# A basic lsdtopotools analysis

We assume you are in our docker container. For this to work you will need to have run the script `install_lsdtt_python_packages.sh` before you start an ipython notebook.

Once you have done that, import some modules:

In [1]:
import lsdviztools.lsdbasemaptools as bmt
from lsdviztools.lsdplottingtools import lsdmap_gdalio as gio
import lsdviztools.lsdmapwrappers as lsdmw
import lsdttparamselector as ps

## Part 1: grab data

First, we need to grab some data. We use a tool in `lsdviztools.lsdbasemaptools` called the `ot_scraper` (`ot` is for opentopography.org).

You can tell this what sort of data you want (most people will use the SRTM 30 metre data) and you also tell it the lower left and the upper right corners, using latitude and longitude. You can get these from goole earth by right clickjing on the map and selecting "what's here". In this example I use a place near Santa Barbara, California. 

In [2]:
SB_DEM = bmt.ot_scraper(source = "SRTM30",longitude_W = -120.464655, longitude_E = -120.254214, latitude_S = 34.440538, latitude_N = 34.610770,prefix = "lg_conception")
SB_DEM.print_parameters()

The source is: SRTM30
The west longitude is: -120.464655
The east longitude is: -120.254214
The south latitude is: 34.61077
The north latitude is: 34.440538
The path is: ./
The prefix is: lg_conception


Okay, we haven't actually got any data. We need to do that by using the `download_pythonic` option. When it is finished it will say `finished downloading`. 

In [3]:
SB_DEM.download_pythonic()

I am going to download the following for you:
https://portal.opentopography.org/otr/getdem?demtype=SRTMGL1&south=34.440538&north=34.61077&west=-120.464655&east=-120.254214&outputFormat=GTiff
This might take a little while, depending on the size of the file. 
Finished downloading


Okay, now we need to do two things. First, we need to convert to UTM. Then we need to convert to ENVI bil format. 

In [4]:
SB_DEM.to_UTM_pythonic()
RasterFile = "lg_conception_SRTM30_UTM.tif"
DataDirectory = "./"
gio.convert2bil(DataDirectory, RasterFile,minimum_elevation=0)

The projections is:
GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]
And some extra projection information strings:
None
WGS 84
{'driver': 'GTiff', 'dtype': 'int16', 'nodata': -32768.0, 'width': 758, 'height': 613, 'count': 1, 'crs': CRS.from_epsg(4326), 'transform': Affine(0.0002777777777778146, 0.0, -120.46486111110323,
       0.0, -0.0002777777777778146, 34.61097222221886)}
res tuple is:
(30.0, 30.0)
The destination CRS is: EPSG:32610
{'driver': 'GTiff', 'dtype': 'int16', 'nodata': -32768.0, 'width': 661, 'height': 647, 'count': 1, 'crs': CRS.from_epsg(32610), 'transform': Affine(30.0, 0.0, 732452.2754693646,
       0.0, -30.0, 3833330.7356094)}


Okay, we now have an ENVI bil DEM. We are ready to start doing topographic analysis!

# Part 2: Topographic analysis

Now, that we have the data, we need to use an `lsdttparamselector` object to select parameters. The current default is to initiate a selector using the command line tool `lsdtt-basic-metrics`.

In [2]:
param = ps.lsdttdm()

You have selected the command line tool: lsdtt-basic-metrics
I am creating a lsdtt-basic-metrics input menu
Okay, your input widget is initiated. Call grab_widget to start entering parameters.
When you have finished, run the read_widgets tool.
This will give you a dictionary that can be passed to an lsdtt_driver object .


Okay, now lets initiate the widget and then call it to enter the parameters we want.

In [3]:
tab_nest = param.make_widget()
tab_nest

Now that we have done that, we grab the parameter values. 

In [4]:
parameter_dict = param.read_widgets()
print(parameter_dict)

{'minimum_elevation': '0.0', 'maximum_elevation': '30000.0', 'min_slope_for_fill': '0.0001', 'raster_is_filled': 'false', 'remove_seas': 'true', 'carve_before_fill': 'false', 'only_check_parameters': 'false', 'write_hillshade': 'true', 'print_raster_without_seas': 'false', 'print_distance_from_outlet': 'false', 'print_fill_raster': 'false', 'print_relief_raster': 'false', 'relief_window': '200.0', 'remove_nodes_influenced_by_edge': 'false', 'isolate_pixels_draining_to_fixed_channel': 'false', 'fixed_channel_csv_name': 'single_channel_nodes', 'print_trimmed_raster': 'false', 'trimming_buffer_pixels': '0', 'convert_csv_to_geojson': 'true', 'burn_raster_to_csv': 'false', 'burn_raster_prefix': 'NULL', 'burn_data_csv_column_header': 'burned_data', 'csv_to_burn_name': 'NULL', 'calculate_window_size': 'false', 'surface_fitting_radius': '60.0', 'print_slope': 'true', 'print_aspect': 'false', 'print_curvature': 'true', 'print_planform_curvature': 'false', 'print_profile_curvature': 'false', 'pr

Okay, now we initiate an `lsdtt_driver` object, which handles the call to `lsdtt-basic-metrics`.

In [8]:
lsdtt_drive = lsdmw.lsdtt_driver(read_prefix = "lg_conception_SRTM30_UTM",write_prefix= "lg_conception_SRTM30_UTM",parameter_dictionary=parameter_dict)
lsdtt_drive.print_parameters()

The command line tool is: lsdtt-basic-metrics
The driver name is: Test_01
The read path is: ./
The write path is: ./
The read prefix is: lg_conception_SRTM30_UTM
The write prefix is: lg_conception_SRTM30_UTM
The parameter dictionary is:
{'minimum_elevation': '0.0', 'maximum_elevation': '30000.0', 'min_slope_for_fill': '0.0001', 'raster_is_filled': 'false', 'remove_seas': 'true', 'carve_before_fill': 'false', 'only_check_parameters': 'false', 'write_hillshade': 'true', 'print_raster_without_seas': 'true', 'print_distance_from_outlet': 'false', 'print_fill_raster': 'false', 'print_relief_raster': 'false', 'relief_window': '200.0', 'remove_nodes_influenced_by_edge': 'false', 'isolate_pixels_draining_to_fixed_channel': 'false', 'fixed_channel_csv_name': 'single_channel_nodes', 'print_trimmed_raster': 'false', 'trimming_buffer_pixels': '0', 'convert_csv_to_geojson': 'false', 'burn_raster_to_csv': 'false', 'burn_raster_prefix': 'NULL', 'burn_data_csv_column_header': 'burned_data', 'csv_to_bu

Now run the tool!

In [9]:
lsdtt_drive.run_lsdtt_command_line_tool()

Done writing the driver file
I've finised writing the driver file. Let me run LSDTT for you.


Super. We will now have a bunch of rasters and csv data ready for visualisation. 

# Part 3: plot some data

Okay, now that we have some rasters, we can use lsdviztools to plot some data. First let's do a hillshade plot. We'll use a `gist_earth` colourmap. This makes an image file (that you can use later). 

In [10]:
DataDirectory = "./"
Base_file = "lg_conception_SRTM30_UTM"
    
img_name = lsdmw.SimpleHillshade(DataDirectory,Base_file,cmap="gist_earth")

Your colourbar will be located: right
xsize: 661 and y size: 647
NoData is: -9999.0
Let me get that projection for you
In this function I will extract the UTM zone
The dataset is projected.
The projection string is: WGS 84 / UTM zone 10N
WGS 84 / UTM zone 10N
This seems to be from the new gdal version
Zone string is: 10N
The zone is: 10
And the hemisphere is: N
The EPSG string is: epsg:32610
epsg:32610
minimum values are x: 732452.0 and y: 3813920.0
I made the ticks.
x labels are: 
['735', '738', '741', '744', '747', '750']
x locations are:
[735000.0, 738000.0, 741000.0, 744000.0, 747000.0, 750000.0]
y labels are: 
['3816', '3819', '3822', '3825', '3828', '3831']
y locations are:
[3816000.0, 3819000.0, 3822000.0, 3825000.0, 3828000.0, 3831000.0]
This colourmap is: gray
The number of axes are: 1
Axes(0,0;1x1)
Axes(0,0;1x1)
N axes are: 1
Axes(0,0;1x1)
xsize: 661 and y size: 647
NoData is: -9999.0
Let me get that projection for you
In this function I will extract the UTM zone
The dataset 

findfont: Font family ['Liberation Sans'] not found. Falling back to DejaVu Sans.


We render the code with a markdown call (it looks like this `<img src="lg_conception_SRTM30_UTM_hillshade.png" width=600>`)

In [1]:
img_name

SyntaxError: invalid syntax (<ipython-input-1-bce3343125f3>, line 1)

<img src="lg_conception_SRTM30_UTM_hillshade.png" width=600>

Now lets print the slope map

In [12]:
Drape_prefix = "lg_conception_SRTM30_UTM_SLOPE"
img_name2 = lsdmw.SimpleDrape(DataDirectory,Base_file, Drape_prefix, cmap = "gnuplot", cbar_loc = "right", cbar_label = "Gradient (m/m)")

Your colourbar will be located: right
xsize: 661 and y size: 647
NoData is: -9999.0
Let me get that projection for you
In this function I will extract the UTM zone
The dataset is projected.
The projection string is: WGS 84 / UTM zone 10N
WGS 84 / UTM zone 10N
This seems to be from the new gdal version
Zone string is: 10N
The zone is: 10
And the hemisphere is: N
The EPSG string is: epsg:32610
epsg:32610
minimum values are x: 732452.0 and y: 3813920.0
I made the ticks.
x labels are: 
['735', '738', '741', '744', '747', '750']
x locations are:
[735000.0, 738000.0, 741000.0, 744000.0, 747000.0, 750000.0]
y labels are: 
['3816', '3819', '3822', '3825', '3828', '3831']
y locations are:
[3816000.0, 3819000.0, 3822000.0, 3825000.0, 3828000.0, 3831000.0]
This colourmap is: gray
The number of axes are: 1
Axes(0,0;1x1)
Axes(0,0;1x1)
N axes are: 1
Axes(0,0;1x1)
xsize: 661 and y size: 647
NoData is: -9999.0
Let me get that projection for you
In this function I will extract the UTM zone
The dataset 

In [13]:
img_name2

'./lg_conception_SRTM30_UTM_drape.png'

<img src="lg_conception_SRTM30_UTM_drape.png" width=600>